<h1 align="center">Pattern Recognition Issue (EEG Signals)</h1>
<h4 align="center">Nikoo Moradi - 400101934</h4>
<h4 align="center">Sharif University of Technology, Fall 2023</h4>
<h4 align="center">Dr. Sepideh Hajipour</h4>
<h4 align="center">AI and Biological Computation Course Project</h4>
<!-- <h5 align="center"><font color="cyan">  </font>
 </h5>
<h5 align="center"> <font color="cyan"> Questions 1,2: @mh_momeni  -  Question 2,3: @Mahdi_h721 </font> </h5> -->

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import periodogram
import statistics

### Loading the Data

In [2]:
data = loadmat('Project_data.mat')

### Extracting our data

In [3]:
# Extract the data
channels = np.array(data['Channels'])
train_data = np.array(data['TrainData'])
train_labels = np.array(data['TrainLabels'])
test_data = np.array(data['TestData'])
sampling_frequency = data['fs']


train_data_pos = train_data[:,:, train_labels[0,:] == 1]
train_data_neg = train_data[:, :, train_labels[0,:] == -1]


print(train_labels.shape)
print(train_data_pos.shape)
print(train_data_neg.shape)

(1, 550)
(59, 5000, 277)
(59, 5000, 273)


### Calculating Fourier Transform on our train data

In [4]:
num_channels = train_data.shape[0]
num_data_points = train_data.shape[1]
num_segments = train_data.shape[2]

fft_train_data = np.zeros((num_channels, num_data_points, num_segments), dtype=complex)

for channel in range(num_channels):
    for segment in range(num_segments):

        segment_data = train_data[channel, :, segment]
        # Apply FFT
        fft_train_data[channel, :, segment] = np.fft.fft(segment_data)


fft_train_data_pos = fft_train_data[:,:, train_labels[0,:] == 1]
fft_train_data_neg = fft_train_data[:, :, train_labels[0,:] == -1]

### Calculating Fourier Transform on our est data

In [5]:
num_channels_test = test_data.shape[0]
num_data_points_test = test_data.shape[1]
num_segments_test = test_data.shape[2]

fft_test_data = np.zeros((num_channels_test, num_data_points_test, num_segments_test), dtype=complex)

for channel in range(num_channels_test):
    for segment in range(num_segments_test):

        segment_data = test_data[channel, :, segment]
        # Apply FFT
        fft_test_data[channel, :, segment] = np.fft.fft(segment_data)

## Frequency domain features

### Calculating the Features , Maximum frequency, Mean frequency and Median frfrequency as well as Power Spectral Ratio for the for bands theta(4,8), alpha(8,13), beta(13,30) and gamma(30,49)

In [6]:
theta_band = (4, 8)
alpha_band = (8, 13)
beta_band = (13, 30)
gamma_band = (30, 49)



max_frequencies = np.zeros((num_channels, num_segments))

mean_frequencies = np.zeros((num_channels, num_segments))

median_frequencies = np.zeros((num_channels, num_segments))

psr_theta = np.zeros((num_channels, num_segments))
psr_alpha = np.zeros((num_channels, num_segments))
psr_beta = np.zeros((num_channels, num_segments))
psr_gamma = np.zeros((num_channels, num_segments))


def band_power(freqs, power_spectrum, band):
    band_power_sum = 0
    for i, freq in enumerate(freqs):
        if band[0] <= freq < band[1]:
            band_power_sum += power_spectrum[i]
    return band_power_sum

def total_power(freqs, power_spectrum):
    tot_power = 0
    for i, freq in enumerate(freqs):
        if 4 <= freq < 49:
            tot_power += power_spectrum[i]
    return tot_power



for channel in range(num_channels):
    for segment in range(num_segments):

        # magnitude 
        magnitude_spectrum = np.abs(fft_train_data[channel, :num_data_points // 2 , segment])

        # PSD and Frequency 
        (freq_values1 , psd) = periodogram(train_data[channel, :, segment], sampling_frequency, scaling = "density")
        temp2 = freq_values1.reshape(2501)

        #total power
        total_power2 = total_power(temp2,psd)


        # Find the frequency with the maximum magnitude
        max_freq_index = np.argmax(magnitude_spectrum)
        max_frequencies[channel, segment] = freq_values1[0,max_freq_index]
        
        # Calculate the mean frequency 
        mean_frequency = np.sum(freq_values1 * psd) / np.sum(psd)
        mean_frequencies[channel, segment] = mean_frequency

        # Compute the cumulative power spectrum
        cumulative_power = np.cumsum(psd)

        # Total power
        total_power3 = cumulative_power[-1]

        # Find the median frequency
        half_power = total_power3 / 2
        median_freq_index = np.where(cumulative_power >= half_power)[0][0]
        median_frequencies[channel, segment] = freq_values1[0,median_freq_index]

        # Calculate PSR for each band
        psr_theta[channel, segment] = band_power(temp2, psd, theta_band) / total_power2
        psr_alpha[channel, segment] = band_power(temp2, psd, alpha_band) / total_power2
        psr_beta[channel, segment] = band_power(temp2, psd, beta_band) / total_power2
        psr_gamma[channel, segment] = band_power(temp2, psd, gamma_band) / total_power2



#### We also calculate test data features

In [7]:
theta_band = (4, 8)
alpha_band = (8, 13)
beta_band = (13, 30)
gamma_band = (30, 49)



max_frequencies_test = np.zeros((num_channels_test, num_segments_test))

mean_frequencies_test = np.zeros((num_channels_test, num_segments_test))

median_frequencies_test = np.zeros((num_channels_test, num_segments_test))

psr_theta_test = np.zeros((num_channels_test, num_segments_test))
psr_alpha_test = np.zeros((num_channels_test, num_segments_test))
psr_beta_test = np.zeros((num_channels_test, num_segments_test))
psr_gamma_test = np.zeros((num_channels_test, num_segments_test))


def band_power(freqs, power_spectrum, band):
    band_power_sum = 0
    for i, freq in enumerate(freqs):
        if band[0] <= freq < band[1]:
            band_power_sum += power_spectrum[i]
    return band_power_sum

def total_power(freqs, power_spectrum):
    tot_power = 0
    for i, freq in enumerate(freqs):
        if 4 <= freq < 49:
            tot_power += power_spectrum[i]
    return tot_power



for channel in range(num_channels_test):
    for segment in range(num_segments_test):

        # magnitude 
        magnitude_spectrum = np.abs(fft_test_data[channel, :num_data_points_test // 2 , segment])

        # PSD and Frequency 
        (freq_values1 , psd) = periodogram(test_data[channel, :, segment], sampling_frequency, scaling = "density")
        temp2 = freq_values1.reshape(2501)

        #total power
        total_power2 = total_power(temp2,psd)


        # Find the frequency with the maximum magnitude
        max_freq_index = np.argmax(magnitude_spectrum)
        max_frequencies_test[channel, segment] = freq_values1[0,max_freq_index]
        
        # Calculate the mean frequency 
        mean_frequency = np.sum(freq_values1 * psd) / np.sum(psd)
        mean_frequencies_test[channel, segment] = mean_frequency

        # Compute the cumulative power spectrum
        cumulative_power = np.cumsum(psd)

        # Total power
        total_power3 = cumulative_power[-1]

        # Find the median frequency
        half_power = total_power3 / 2
        median_freq_index = np.where(cumulative_power >= half_power)[0][0]
        median_frequencies_test[channel, segment] = freq_values1[0,median_freq_index]

        # Calculate PSR for each band
        psr_theta_test[channel, segment] = band_power(temp2, psd, theta_band) / total_power2
        psr_alpha_test[channel, segment] = band_power(temp2, psd, alpha_band) / total_power2
        psr_beta_test[channel, segment] = band_power(temp2, psd, beta_band) / total_power2
        psr_gamma_test[channel, segment] = band_power(temp2, psd, gamma_band) / total_power2



#### We calculated the maximum frequency of each channel and each segment. Now we calculate average of this feature on all of the data, positive labels and negetive labels, as well as its variance for positive and negetive labels; in order to use the Fisher method

In [8]:
def Fisher(feature, train_labels):
    avg_feature = np.mean(feature , axis=1)

    avg_feature_pos = np.mean(feature[:, train_labels[0,:] == 1],axis = 1)
    avg_feature_neg = np.mean(feature[:, train_labels[0,:] == -1],axis = 1)

    var_feature_pos = np.var(feature[:, train_labels[0,:] == 1],axis = 1)
    var_feature_neg = np.var(feature[:, train_labels[0,:] == -1],axis = 1)

    fisher = ((avg_feature_pos - avg_feature)**2 + (avg_feature_neg - avg_feature)**2)/(var_feature_pos + var_feature_neg)

    return fisher

#### We use the Min-Max normalization method before we apply Fisher on out features

In [9]:
from sklearn.preprocessing import MinMaxScaler

def normalize(feature):

    scaler = MinMaxScaler()
    normalized_feature = scaler.fit_transform(feature.T)
    
    return normalized_feature.T

In [10]:
### Max freq

# Normalizing 
max_frequencies_test = normalize(max_frequencies_test)
max_frequencies = normalize(max_frequencies)

# Fisher for max frequency per channel
print(max_frequencies.shape)
F_max_freq_per_channel = Fisher(max_frequencies,train_labels)

print("Fisher Method for maximum frequency : \n",np.around(F_max_freq_per_channel,4),"\n\n")


(59, 550)
Fisher Method for maximum frequency : 
 [0.0011 0.     0.0003 0.0009 0.0042 0.0003 0.003  0.0076 0.0021 0.0002
 0.0002 0.0015 0.0005 0.0049 0.0001 0.0025 0.003  0.0034 0.0038 0.0004
 0.0103 0.0038 0.0224 0.0237 0.0016 0.0001 0.0016 0.     0.0002 0.0089
 0.0005 0.0037 0.0038 0.0069 0.0059 0.0009 0.0036 0.0014 0.0039 0.0141
 0.     0.     0.0234 0.0002 0.0005 0.     0.0003 0.0027 0.0012 0.0063
 0.0167 0.0004 0.0002 0.0172 0.0013 0.0016 0.0011 0.0036 0.0007] 




In [11]:
### Mean freq

# Normalizing
mean_frequencies_test = normalize(mean_frequencies_test)
mean_frequencies = normalize(mean_frequencies)

# Fisher for Mean frequency per channel
F_mean_freq_per_channel = Fisher(mean_frequencies,train_labels)

print("Fisher Method for mean frequency : \n",np.around(F_mean_freq_per_channel,4),"\n\n")


Fisher Method for mean frequency : 
 [0.     0.0009 0.0009 0.     0.0013 0.0047 0.0022 0.0002 0.0002 0.0001
 0.     0.0002 0.0039 0.0011 0.0003 0.0015 0.0052 0.0003 0.     0.0053
 0.0029 0.0079 0.0245 0.0048 0.0016 0.0082 0.0003 0.     0.0014 0.003
 0.0109 0.0093 0.0189 0.006  0.0004 0.0025 0.0006 0.0017 0.006  0.0063
 0.0024 0.006  0.011  0.0029 0.0013 0.0015 0.0082 0.0028 0.0087 0.007
 0.0008 0.0009 0.0087 0.0103 0.0038 0.0144 0.0068 0.     0.0179] 




In [12]:
### median freq

# Normalizing
median_frequencies_test = normalize(median_frequencies_test)
median_frequencies = normalize(median_frequencies)

# Fisher for median frequency per channel
F_med_freq_per_channel = Fisher(median_frequencies,train_labels)

print("Fisher Method for median frequency : \n",np.around(F_med_freq_per_channel,4),"\n\n")

Fisher Method for median frequency : 
 [0.     0.0003 0.0008 0.     0.0006 0.0021 0.0006 0.0005 0.0001 0.
 0.0001 0.0002 0.0048 0.0034 0.0003 0.0014 0.0055 0.0002 0.0002 0.0054
 0.0051 0.0086 0.0203 0.0055 0.0003 0.0118 0.0005 0.0003 0.     0.0012
 0.0097 0.0054 0.0197 0.0028 0.0045 0.0007 0.     0.0035 0.0029 0.0055
 0.001  0.0075 0.0313 0.0007 0.0026 0.001  0.0164 0.0026 0.0099 0.0045
 0.0012 0.0001 0.0104 0.0194 0.0094 0.0201 0.0067 0.0001 0.0195] 




In [13]:
### PSR for theta band

# Normalizing
psr_theta_test = normalize(psr_theta_test)
psr_theta = normalize(psr_theta)

# Fisher for PSR for theta band per channel
F_psr_theta_per_channel =  Fisher(psr_theta,train_labels)

print("Fisher Method for theta psr : \n",np.around(F_psr_theta_per_channel,4),"\n\n")


Fisher Method for theta psr : 
 [0.0002 0.0028 0.0001 0.001  0.0034 0.0042 0.0014 0.0011 0.     0.0053
 0.0001 0.0013 0.0078 0.0003 0.0043 0.0002 0.0099 0.0007 0.0076 0.0043
 0.0008 0.0022 0.0248 0.006  0.0072 0.0053 0.0027 0.0044 0.0004 0.
 0.0113 0.0103 0.0113 0.0109 0.0016 0.0021 0.     0.0009 0.0051 0.009
 0.0063 0.0164 0.0014 0.     0.0006 0.0025 0.0114 0.009  0.0134 0.0358
 0.0067 0.0173 0.0077 0.0095 0.01   0.0117 0.0357 0.0078 0.0255] 




In [14]:
### PSR for alpha band

# Normalizing 
psr_alpha_test = normalize(psr_alpha_test)
psr_alpha = normalize(psr_alpha)

# Fisher for PSR for alpha per channel
F_psr_alpha_per_channel = Fisher(psr_alpha,train_labels)

print("Fisher Method for alpha psr : \n",np.around(F_psr_alpha_per_channel,4),"\n\n")


Fisher Method for alpha psr : 
 [0.0014 0.0026 0.     0.0008 0.0012 0.0001 0.0018 0.0007 0.0001 0.0007
 0.0001 0.0001 0.0013 0.0008 0.0003 0.0003 0.0074 0.0032 0.008  0.0019
 0.0068 0.003  0.0072 0.0033 0.0027 0.0103 0.0042 0.0093 0.0021 0.0048
 0.0048 0.0051 0.005  0.0017 0.0029 0.0088 0.0031 0.003  0.0017 0.0034
 0.0032 0.0035 0.0077 0.0035 0.0047 0.0008 0.0045 0.0017 0.0043 0.0001
 0.0008 0.001  0.0039 0.0042 0.0028 0.0054 0.     0.     0.0067] 




In [15]:
### PSR for beta band

# Normalizing
psr_beta_test = normalize(psr_beta_test)
psr_beta = normalize(psr_beta)

# Fisher for PSR for beta per channel
F_psr_beta_per_channel = Fisher(psr_beta,train_labels)

print("Fisher Method for beta psr : \n",np.around(F_psr_beta_per_channel,4),"\n\n")


Fisher Method for beta psr : 
 [0.001  0.0039 0.     0.0045 0.0042 0.0003 0.0065 0.0004 0.     0.0025
 0.0003 0.0017 0.0043 0.     0.0021 0.0004 0.     0.0004 0.0005 0.0016
 0.0023 0.0023 0.0128 0.0048 0.0082 0.0046 0.0021 0.0055 0.0026 0.0068
 0.005  0.0105 0.0009 0.0073 0.0024 0.0076 0.0019 0.0059 0.0012 0.0078
 0.0146 0.0061 0.0149 0.0005 0.0046 0.0002 0.0036 0.0021 0.0055 0.0147
 0.009  0.0123 0.003  0.0023 0.005  0.0032 0.0146 0.0001 0.0073] 




In [16]:
### PSR for gamma band

# Normalizing
psr_gamma_test = normalize(psr_gamma_test)
psr_gamma = normalize(psr_gamma)

# Fisher for PSR for gamma per channel
F_psr_gamma_per_channel = Fisher(psr_gamma,train_labels)

print("Fisher Method for gamma psr : \n",np.around(F_psr_gamma_per_channel,4),"\n\n")

Fisher Method for gamma psr : 
 [0.0002 0.0003 0.0001 0.0001 0.0015 0.0008 0.0001 0.     0.     0.0002
 0.0007 0.0001 0.0034 0.001  0.0012 0.0002 0.0013 0.0007 0.0003 0.0043
 0.0021 0.0031 0.0186 0.0043 0.0033 0.0039 0.0016 0.0035 0.0035 0.0037
 0.011  0.0099 0.0171 0.006  0.0006 0.0079 0.0051 0.0048 0.0062 0.0122
 0.0023 0.0181 0.0022 0.0093 0.0084 0.004  0.0132 0.0063 0.0159 0.0249
 0.0035 0.0111 0.01   0.0154 0.0069 0.0174 0.0107 0.0018 0.0257] 




## Time domain features

In [17]:
def calculate_form_factor(segment_data):

    first_derivation = np.diff(segment_data)
    second_derivation = np.diff(first_derivation)

    std_first_diff = np.std(first_derivation)
    std_sec_diff = np.std(second_derivation)
    std_signal = np.std(segment_data)

    if std_signal != 0:
        form_factor = (std_signal * std_sec_diff) / (std_first_diff)**2
    else:
        form_factor = 0

    return form_factor

In [18]:
def calculate_correlation(data):

    num_channels, _, num_segments = data.shape

    corrrelations = np.zeros((num_channels , num_channels, num_segments ))

    for segment in range(num_segments):
        for channel in range(num_channels):
            for other_channel in range(num_channels):

                correlation_coefficient = np.corrcoef(data[channel, :, segment], data[other_channel, :, segment])[0, 1]

                corrrelations[channel, other_channel, segment] = correlation_coefficient

    return corrrelations


In [19]:
import numpy as np
variances = np.zeros((num_channels, num_segments))

num_bins = 10
amp_histograms = np.zeros((num_channels, num_segments, num_bins))

form_factors = np.zeros((num_channels, num_segments))



for channel in range(num_channels):
    for segment in range(num_segments):

        segment_data = train_data[channel, :, segment]

        # Variance
        variances[channel, segment] = np.var(segment_data)

        # Amplitude histogram
        counts, bin_edges = np.histogram(segment_data,num_bins, density= False)
        amp_histograms[channel, segment, :] = counts

        # Form Factor
        form_factors[channel, segment] = calculate_form_factor(segment_data)

    

In [20]:
# Correlation
correlations = calculate_correlation(train_data)

#### We also calculate these features for our test data

In [21]:
variances_test = np.zeros((num_channels_test, num_segments_test))

num_bins = 10
amp_histograms_test = np.zeros((num_channels_test, num_segments_test, num_bins))

form_factors_test = np.zeros((num_channels_test, num_segments_test))



for channel in range(num_channels_test):
    for segment in range(num_segments_test):

        segment_data = train_data[channel, :, segment]

        # Variance
        variances_test[channel, segment] = np.var(segment_data)

        # Amplitude histogram
        counts, bin_edges = np.histogram(segment_data,num_bins, density= False)
        amp_histograms_test[channel, segment, :] = counts

        # Form Factor
        form_factors_test[channel, segment] = calculate_form_factor(segment_data)

    

In [22]:
# Correlation
correlations_test = calculate_correlation(test_data)

### We calculated time domain features such as form factor, correlation between every tow channels, variance and amplitude histogram. Now we calculate Fisher for each feature.

In [23]:
### Amplitude Histogram

# Normalizing
for bin in range(num_bins):
    amp_histograms[:,:,bin] = normalize(amp_histograms[:,:,bin])
    amp_histograms_test[:,:,bin] = normalize(amp_histograms_test[:,:,bin])


# Fisher for Amplitude Histogram per channel
avg_feature = np.mean(amp_histograms, axis=1)

avg_feature_pos = np.mean(amp_histograms[:, train_labels[0,:] == 1 , :],axis = 1)
avg_feature_neg = np.mean(amp_histograms[:, train_labels[0,:] == -1 , :],axis = 1)

var_feature_pos = np.var(amp_histograms[:, train_labels[0,:] == 1 , :],axis = 1)
var_feature_neg = np.var(amp_histograms[:, train_labels[0,:] == -1 , :],axis = 1)

fisher = ((avg_feature_pos - avg_feature)**2 + (avg_feature_neg - avg_feature)**2)/(var_feature_pos + var_feature_neg)
F_amp_hist_per_channel = fisher

print("Fisher Method for Amplitude Histogram : \n",np.around(F_amp_hist_per_channel,4),"\n\n")
print(F_amp_hist_per_channel.shape)

Fisher Method for Amplitude Histogram : 
 [[0.013  0.0098 0.0085 0.0052 0.0037 0.0048 0.0068 0.0064 0.0117 0.0119]
 [0.0022 0.0058 0.0075 0.0056 0.0011 0.0053 0.0045 0.0031 0.0003 0.0002]
 [0.0034 0.0009 0.0001 0.0001 0.0009 0.0032 0.     0.0027 0.0014 0.0004]
 [0.0057 0.     0.001  0.004  0.0056 0.0027 0.0027 0.0028 0.0029 0.0026]
 [0.0001 0.0001 0.0005 0.0004 0.0005 0.0032 0.0001 0.0002 0.0001 0.0041]
 [0.0013 0.0067 0.0079 0.0021 0.0003 0.0001 0.0022 0.0042 0.0041 0.0008]
 [0.0109 0.0061 0.0028 0.002  0.0016 0.0003 0.0029 0.005  0.0018 0.0004]
 [0.     0.0006 0.0001 0.     0.     0.0001 0.0001 0.0002 0.0001 0.0042]
 [0.0011 0.0129 0.0237 0.0199 0.0015 0.0179 0.0194 0.0107 0.0058 0.003 ]
 [0.001  0.0007 0.0013 0.0001 0.0006 0.001  0.     0.     0.     0.    ]
 [0.0073 0.0209 0.0324 0.0192 0.0031 0.0251 0.0155 0.0099 0.003  0.0018]
 [0.0008 0.0026 0.0032 0.0004 0.0004 0.0007 0.0018 0.0044 0.0029 0.0035]
 [0.0079 0.0002 0.0011 0.0039 0.0016 0.0005 0.0043 0.0074 0.0069 0.0077]
 [0.0011 

In [24]:
### Variance

# Normalizing 
variances_test = normalize(variances_test)
variances = normalize(variances)

# Fisher for Variance per channel
F_variance_per_channel = Fisher(variances, train_labels)

print("Fisher Method for Variance : \n",np.around(F_variance_per_channel,4),"\n\n")

Fisher Method for Variance : 
 [0.     0.0017 0.0032 0.0009 0.0061 0.0102 0.0105 0.0018 0.0038 0.0074
 0.0016 0.0048 0.0133 0.0027 0.0125 0.0053 0.002  0.0039 0.0035 0.003
 0.0018 0.0002 0.0034 0.0024 0.0036 0.0084 0.0025 0.0038 0.0021 0.0012
 0.     0.0017 0.0049 0.003  0.0092 0.0085 0.0048 0.0019 0.0008 0.0002
 0.0009 0.004  0.0157 0.0069 0.0016 0.0063 0.0014 0.0044 0.0001 0.0107
 0.0026 0.0053 0.0113 0.     0.0047 0.0001 0.0069 0.0005 0.001 ] 




In [25]:
### Form Factor

# Normalizing 
form_factors_test = normalize(form_factors_test)
form_factors = normalize(form_factors)

# Fisher for Form Factor per channel
F_form_factors_per_channel = Fisher(form_factors, train_labels)

print("Fisher Method for Form Factor : \n",np.around(F_form_factors_per_channel,4),"\n\n")

Fisher Method for Form Factor : 
 [0.0023 0.0007 0.005  0.0005 0.0004 0.0027 0.0004 0.0008 0.0008 0.0001
 0.0006 0.     0.002  0.0016 0.0002 0.0006 0.002  0.     0.     0.0003
 0.     0.0001 0.0053 0.0005 0.0021 0.0034 0.     0.     0.     0.0002
 0.0014 0.0026 0.0006 0.0035 0.     0.0017 0.0018 0.0002 0.0012 0.0016
 0.0024 0.0029 0.     0.0005 0.     0.0008 0.0005 0.0009 0.0018 0.0005
 0.001  0.0006 0.0012 0.0005 0.     0.0006 0.     0.0015 0.0049] 




In [26]:
### Correlation 

# Normalizing 
for channel in range(num_channels):
            correlations[channel,:,:] = normalize(correlations[channel,:,:] )
            correlations_test[channel,:,:] = normalize(correlations_test[channel,:,:])
 
# Fisher for Correlation per channel
avg_feature = np.mean(correlations, axis=2)

avg_feature_pos = np.mean(correlations[:, :, train_labels[0,:] == 1], axis = 2)
avg_feature_neg = np.mean(correlations[:, :, train_labels[0,:] == -1], axis = 2)

var_feature_pos = np.var(correlations[:, :, train_labels[0,:] == 1], axis = 2)
var_feature_neg = np.var(correlations[:, :, train_labels[0,:] == -1], axis = 2)

fisher = ((avg_feature_pos - avg_feature)**2 + (avg_feature_neg - avg_feature)**2)/(var_feature_pos + var_feature_neg)
F_corr_per_channel = fisher

print("Fisher Method for Correlation : \n",np.around(F_corr_per_channel,4),"\n\n")
print(F_corr_per_channel.shape)

Fisher Method for Correlation : 
 [[0.0008 0.0002 0.0001 ... 0.0008 0.003  0.0005]
 [0.0002 0.0066 0.0014 ... 0.0022 0.0036 0.0013]
 [0.0001 0.0014 0.0062 ... 0.0002 0.0023 0.0005]
 ...
 [0.0008 0.0022 0.0002 ... 0.0009 0.0111 0.    ]
 [0.003  0.0036 0.0023 ... 0.0111 0.0005 0.0013]
 [0.0005 0.0013 0.0005 ... 0.     0.0013 0.0011]] 


(59, 59)


# Feature Selection

In [27]:
# Combine scores with their channel and feature identifiers

combined_fisher = []
test_features = []

for channel in range(num_channels):
    combined_fisher.append((f"Max Freq _ ch {channel+1}", F_max_freq_per_channel[channel], max_frequencies[channel, :], max_frequencies_test[channel, :]))
    combined_fisher.append((f"Mean Freq _ ch {channel+1}", F_mean_freq_per_channel[channel], mean_frequencies[channel, :], mean_frequencies_test[channel, :]))
    combined_fisher.append((f"Med Freq _ ch {channel+1}", F_med_freq_per_channel[channel], median_frequencies[channel, :], median_frequencies_test[channel, :]))
    combined_fisher.append((f"Power ratio (theta) _ ch {channel+1}", F_psr_theta_per_channel[channel], psr_theta[channel, :], psr_theta_test[channel, :]))
    combined_fisher.append((f"Power ratio (alpha) _ ch {channel+1}", F_psr_alpha_per_channel[channel], psr_alpha[channel, :], psr_alpha_test[channel, :]))
    combined_fisher.append((f"Power ratio (beta) _ ch {channel+1}", F_psr_beta_per_channel[channel], psr_beta[channel, :], psr_beta_test[channel, :]))
    combined_fisher.append((f"Power ratio (gamma) _ ch {channel+1}", F_psr_gamma_per_channel[channel], psr_gamma[channel, :], psr_gamma_test[channel, :]))
    combined_fisher.append((f"Variance _ ch {channel+1}", F_variance_per_channel[channel], variances[channel, :], variances_test[channel, :]))
    combined_fisher.append((f"Form Factor _ ch {channel+1}", F_form_factors_per_channel[channel], form_factors[channel, :], form_factors_test[channel, :]))


    for bin_idx in range(num_bins):  
        score = F_amp_hist_per_channel[channel, bin_idx]
        feature_name = f"histogram_amp _ ch {channel+1} _ bin {bin_idx+1}"
        feature_temp = amp_histograms[channel, :, bin_idx]
        combined_fisher.append((feature_name, score, feature_temp, amp_histograms_test[channel, :, bin_idx]))


for channel in range(num_channels):
    for other_channel in range(channel+1, num_channels):
        score = F_corr_per_channel[channel, other_channel]
        feature_name = f"correlation {channel+1}_{other_channel+1}" 
        feature_temp = correlations[channel, other_channel, :]
        combined_fisher.append((feature_name, score, feature_temp, correlations_test[channel, other_channel, :]))

# store all of the features in an array
all_features = np.array([feature[2] for feature in combined_fisher]).T
all_fisher = np.array([feature[1] for feature in combined_fisher]).T

print(all_features.shape)
# Sort the combined list by scores in descending order
combined_fisher.sort(key=lambda x: x[1], reverse=True)

# Select the top 70 features
top_70_features = combined_fisher[:70]

# Print the selected features and their channels
for feature in top_70_features:
    print(f"Feature: {feature[0]}\nScore: {feature[1]}\nfeature: {feature[2].shape}\n")
    


(550, 2832)
Feature: Power ratio (theta) _ ch 50
Score: 0.03582588658785932
feature: (550,)

Feature: Power ratio (theta) _ ch 57
Score: 0.03567267674268758
feature: (550,)

Feature: histogram_amp _ ch 11 _ bin 3
Score: 0.03240029904844514
feature: (550,)

Feature: Med Freq _ ch 43
Score: 0.031301709484047764
feature: (550,)

Feature: histogram_amp _ ch 20 _ bin 4
Score: 0.025724108421415107
feature: (550,)

Feature: Power ratio (gamma) _ ch 59
Score: 0.025696274717266175
feature: (550,)

Feature: Power ratio (theta) _ ch 59
Score: 0.025491696411105512
feature: (550,)

Feature: histogram_amp _ ch 11 _ bin 6
Score: 0.0250597878877978
feature: (550,)

Feature: Power ratio (gamma) _ ch 50
Score: 0.024934922771357487
feature: (550,)

Feature: Power ratio (theta) _ ch 23
Score: 0.024796059296197194
feature: (550,)

Feature: Mean Freq _ ch 23
Score: 0.024532674098154097
feature: (550,)

Feature: histogram_amp _ ch 49 _ bin 4
Score: 0.024005537944502817
feature: (550,)

Feature: Max Freq _ ch

### Now that we extracted the best 50 features, first we should build our feature matrix

In [28]:
# Feature Matrix

features_matrix = np.array([feature[2] for feature in top_70_features]).T

features_matrix_test = np.array([feature[3] for feature in top_70_features]).T

print(features_matrix_test.shape)
print("X_train shape : ",features_matrix.shape)

# Lables

y_train = (np.where(train_labels == -1, 0, 1)).reshape(550)



(159, 70)
X_train shape :  (550, 70)


# MLP network

### After building our feature matrix, we need to design and train our MLP network. I used Tenserflow which is a strong library for building and implementing neural networks such as MLP. My network has 3 hidden layers, each with 128, 64 and 32 neurons respectively. The activation functions of the first and second hidden layers are relu and the last hidden layer is sigmoid.


In [29]:
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
from itertools import combinations
import random

In [30]:
# Parameters
n_splits = 5
n_epochs = 100
batch_size = 32
n_features = 50  
accuracy_mlp = 0
acc_train = 0

# Define various MLP models
def create_mlp_model(layer_neurons,n_features):
    model = Sequential() 
    model.add(Dense(layer_neurons[0], input_dim = n_features, activation = 'relu'))

    for i in range(1,len(layer_neurons)):
        if i == len(layer_neurons)-1:
            model.add(Dense(layer_neurons[i], activation = 'sigmoid'))  # Output layer

        else:
            model.add(Dense(layer_neurons[i], activation = 'relu'))  # 2nd hidden layer

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [31]:
layers = [[128,64,32,1],[64,16,1]]
mlp_models_fisher = []

for i,layer_neurons in enumerate(layers):
    for _ in range(3):

        accuracy_mlp=0
        selected_indices = random.sample(range(70), 50)
        selected_features = [top_70_features[i][0] for i in selected_indices]

        X_train_fisher = features_matrix[:, selected_indices]
        model_mlp = create_mlp_model(layer_neurons, n_features)

        # K-Fold 
        kf = KFold(n_splits = n_splits, shuffle = True)
        fold_no = 1

        for train_indeX, test_indeX in kf.split(X_train_fisher, y_train):

            X_train_train, X_train_val = X_train_fisher[train_indeX], X_train_fisher[test_indeX]
            y_train_train, y_train_val = y_train[train_indeX], y_train[test_indeX]


            print(f'Training for fold {fold_no} ...')
            model_mlp.fit(X_train_train, y_train_train, epochs = n_epochs, batch_size = batch_size)


            scores = model_mlp.evaluate(X_train_val, y_train_val, verbose=0)

            print(f'Score for fold {fold_no}: {model_mlp.metrics_names[1]} of {scores[1]*100}%')
            fold_no += 1
        
            accuracy_mlp += scores[1]*100
        

        mlp_models_fisher.append((layer_neurons, selected_features, accuracy_mlp/n_splits, selected_indices, model_mlp))



Training for fold 1 ...
Epoch 1/100
14/14 [==============================] - 0s 880us/step - loss: 0.6693 - accuracy: 0.5932
Epoch 2/100
14/14 [==============================] - 0s 794us/step - loss: 0.6551 - accuracy: 0.6250
Epoch 3/100
14/14 [==============================] - 0s 752us/step - loss: 0.6311 - accuracy: 0.6455
Epoch 4/100
14/14 [==============================] - 0s 806us/step - loss: 0.6151 - accuracy: 0.6659
Epoch 5/100
14/14 [==============================] - 0s 720us/step - loss: 0.6053 - accuracy: 0.6795
Epoch 6/100
14/14 [==============================] - 0s 748us/step - loss: 0.5851 - accuracy: 0.6955
Epoch 7/100
14/14 [==============================] - 0s 691us/step - loss: 0.5663 - accuracy: 0.7136
Epoch 8/100
14/14 [==============================] - 0s 880us/step - loss: 0.5608 - accuracy: 0.7091
Epoch 9/100
14/14 [==============================] - 0s 907us/step - loss: 0.5422 - accuracy: 0.7341
Epoch 10/100
14/14 [==============================] - 0s 752us/step

In [33]:
mlp_models_fisher.sort(key=lambda x: x[2], reverse=True)

for temp in mlp_models_fisher:
    print(f"Layer neurons: {temp[0]}\nFeatures: {len(temp[1])}\nAccurcy: {np.round(temp[2],2)}%\n")

Layer neurons: [128, 64, 32, 1]
Features: 50
Accurcy: 95.09%

Layer neurons: [128, 64, 32, 1]
Features: 50
Accurcy: 94.36%

Layer neurons: [128, 64, 32, 1]
Features: 50
Accurcy: 93.82%

Layer neurons: [64, 16, 1]
Features: 50
Accurcy: 89.64%

Layer neurons: [64, 16, 1]
Features: 50
Accurcy: 89.64%

Layer neurons: [64, 16, 1]
Features: 50
Accurcy: 88.55%



In [34]:
import pandas as pd

Best_mlp_fisher = mlp_models_fisher[0]
print(f"Best MLP Model :\n\nLayer neurons: {Best_mlp_fisher[0]}\n\nFeatures: {Best_mlp_fisher[1]}\n\nnumber of features: {len(Best_mlp_fisher[3])}\n\nAccurcy: {np.round(Best_mlp_fisher[2],2)}%\n")

df = pd.DataFrame(Best_mlp_fisher[1], columns=['features_fisher_mlp'])
df.to_csv("slected_features_mlp_fisher.csv", index=False)

Best MLP Model :

Layer neurons: [128, 64, 32, 1]

Features: ['correlation 40_56', 'Power ratio (beta) _ ch 57', 'histogram_amp _ ch 11 _ bin 3', 'correlation 55_58', 'Max Freq _ ch 24', 'Med Freq _ ch 59', 'Power ratio (gamma) _ ch 54', 'Power ratio (gamma) _ ch 50', 'correlation 49_56', 'Power ratio (gamma) _ ch 56', 'Max Freq _ ch 51', 'Mean Freq _ ch 56', 'Power ratio (theta) _ ch 50', 'Power ratio (gamma) _ ch 23', 'Power ratio (beta) _ ch 50', 'Med Freq _ ch 23', 'correlation 39_52', 'Power ratio (beta) _ ch 43', 'histogram_amp _ ch 11 _ bin 2', 'correlation 48_52', 'correlation 40_58', 'histogram_amp _ ch 20 _ bin 4', 'Max Freq _ ch 43', 'correlation 39_59', 'Power ratio (gamma) _ ch 59', 'histogram_amp _ ch 20 _ bin 6', 'histogram_amp _ ch 9 _ bin 3', 'histogram_amp _ ch 20 _ bin 3', 'histogram_amp _ ch 25 _ bin 5', 'Max Freq _ ch 23', 'histogram_amp _ ch 49 _ bin 6', 'Power ratio (theta) _ ch 59', 'histogram_amp _ ch 9 _ bin 7', 'Med Freq _ ch 47', 'correlation 34_56', 'Med Fr

# Now we predict our test data labels using the best MLP network found with the Fisher method

In [77]:
import pandas as pd

X_test = features_matrix_test[:, Best_mlp_fisher[3]]
print(X_test.shape)
y_pred_mlp = Best_mlp_fisher[4].predict(X_test)

print("Predicted Labels for test set by the best MLP model (Fisher) :\n",np.round(y_pred_mlp).T)
print(y_pred_mlp.shape)

df = pd.DataFrame(np.round(y_pred_mlp), columns=['y_pred_mlp_fisher'])
df.to_csv("y_pred_mlp_fisher.csv", index=False)

(159, 50)
5/5 [==============================] - 0s 927us/step
Predicted Labels for test set by the best MLP model (Fisher) :
 [[0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1.
  0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.
  1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0.
  1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
  1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1.
  1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1.
  1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1.]]
(159, 1)


# RBF Network

In [36]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def rbf(num_centers):

    selected_indices = random.sample(range(70), 50)
    selected_features = [top_70_features[i][0] for i in selected_indices]

    X_train_fisher1 = features_matrix[:, selected_indices]

    kfold = KFold(n_splits=n_splits, shuffle=True)
    fold_no = 1
    accuracy_rbf = 0
    svm_classifier = SVC(kernel='rbf', C=num_centers, gamma='scale')  

    # for train_indeX, test_indeX in kf.split(X_train_fisher):
    for train_indeX, test_indeX in kfold.split(X_train_fisher1, y_train):
        X_train_train, X_train_val = X_train_fisher1[train_indeX], X_train_fisher1[test_indeX]
        y_train_train, y_train_val = y_train[train_indeX], y_train[test_indeX]

        # Fit the SVM classifier
        svm_classifier.fit(X_train_train, y_train_train)

        # Predict on the test set
        y_pred = svm_classifier.predict(X_train_val)

        # Calculate accuracy (you can use other metrics as needed)
        scores = accuracy_score(y_train_val, y_pred)
        # print(f'Accuracy for fold {fold_no}: {scores * 100} %')
        accuracy_rbf += (scores * 100)
        fold_no += 1

    # Calculate average accuracy across folds
    avg_accuracy_rbf = accuracy_rbf / n_splits
    print(f'Average Accuracy across 5-fold Cross Validation for RBF network ({num_centers}): {np.round(avg_accuracy_rbf,2)} %')
    
    return ((num_centers, selected_features, avg_accuracy_rbf, selected_indices, svm_classifier))


In [37]:
import matplotlib.pyplot as plt

num_centers = [16,32,64,128,256,512]
rbf_models_fisher = []

for i,num_center in enumerate(num_centers): 
    for j in range(3):
        avg_accuracy_rbf = rbf(num_center)
        rbf_models_fisher.append(avg_accuracy_rbf)


Average Accuracy across 5-fold Cross Validation for RBF network (16): 76.0 %
Average Accuracy across 5-fold Cross Validation for RBF network (16): 78.36 %
Average Accuracy across 5-fold Cross Validation for RBF network (16): 76.91 %
Average Accuracy across 5-fold Cross Validation for RBF network (32): 79.82 %
Average Accuracy across 5-fold Cross Validation for RBF network (32): 78.91 %
Average Accuracy across 5-fold Cross Validation for RBF network (32): 77.82 %
Average Accuracy across 5-fold Cross Validation for RBF network (64): 74.73 %
Average Accuracy across 5-fold Cross Validation for RBF network (64): 78.91 %
Average Accuracy across 5-fold Cross Validation for RBF network (64): 75.09 %
Average Accuracy across 5-fold Cross Validation for RBF network (128): 74.36 %
Average Accuracy across 5-fold Cross Validation for RBF network (128): 77.09 %
Average Accuracy across 5-fold Cross Validation for RBF network (128): 77.82 %
Average Accuracy across 5-fold Cross Validation for RBF networ

In [38]:
rbf_models_fisher.sort(key=lambda x: x[2], reverse=True)

for temp in rbf_models_fisher:
    print(f"Hidden Layer Neuros: {temp[0]}\nFeatures: {len(temp[1])}\nAccurcy: {np.round(temp[2],2)}\n")

Hidden Layer Neuros: 32
Features: 50
Accurcy: 79.82

Hidden Layer Neuros: 32
Features: 50
Accurcy: 78.91

Hidden Layer Neuros: 64
Features: 50
Accurcy: 78.91

Hidden Layer Neuros: 16
Features: 50
Accurcy: 78.36

Hidden Layer Neuros: 256
Features: 50
Accurcy: 78.18

Hidden Layer Neuros: 32
Features: 50
Accurcy: 77.82

Hidden Layer Neuros: 128
Features: 50
Accurcy: 77.82

Hidden Layer Neuros: 128
Features: 50
Accurcy: 77.09

Hidden Layer Neuros: 16
Features: 50
Accurcy: 76.91

Hidden Layer Neuros: 256
Features: 50
Accurcy: 76.91

Hidden Layer Neuros: 512
Features: 50
Accurcy: 76.91

Hidden Layer Neuros: 512
Features: 50
Accurcy: 76.73

Hidden Layer Neuros: 512
Features: 50
Accurcy: 76.36

Hidden Layer Neuros: 16
Features: 50
Accurcy: 76.0

Hidden Layer Neuros: 64
Features: 50
Accurcy: 75.09

Hidden Layer Neuros: 256
Features: 50
Accurcy: 74.91

Hidden Layer Neuros: 64
Features: 50
Accurcy: 74.73

Hidden Layer Neuros: 128
Features: 50
Accurcy: 74.36



In [39]:
import pandas as pd

Best_rbf_fisher = rbf_models_fisher[0]
print(f"Best RBF Model :\n\nLayer neurons: {Best_rbf_fisher[0]}\n\nFeatures: {Best_rbf_fisher[1]}\n\nnumber of features: {len(Best_rbf_fisher[3])}\n\nAccurcy: {np.round(Best_rbf_fisher[2],2)}%\n")

df = pd.DataFrame(Best_rbf_fisher[1], columns=['features_fisher_mlp'])
df.to_csv("selected_features_rbf_fisher.csv", index=False)

Best RBF Model :

Layer neurons: 32

Features: ['Max Freq _ ch 54', 'Power ratio (theta) _ ch 50', 'Max Freq _ ch 51', 'correlation 48_52', 'correlation 42_56', 'Med Freq _ ch 23', 'Mean Freq _ ch 59', 'Variance _ ch 43', 'histogram_amp _ ch 20 _ bin 7', 'correlation 37_59', 'histogram_amp _ ch 11 _ bin 4', 'correlation 44_59', 'Med Freq _ ch 43', 'Mean Freq _ ch 23', 'Med Freq _ ch 47', 'Power ratio (theta) _ ch 42', 'correlation 41_52', 'correlation 39_59', 'histogram_amp _ ch 20 _ bin 4', 'Med Freq _ ch 56', 'histogram_amp _ ch 9 _ bin 3', 'histogram_amp _ ch 9 _ bin 7', 'correlation 13_53', 'histogram_amp _ ch 25 _ bin 5', 'Power ratio (beta) _ ch 41', 'Power ratio (theta) _ ch 57', 'histogram_amp _ ch 49 _ bin 3', 'Mean Freq _ ch 33', 'Power ratio (beta) _ ch 43', 'Power ratio (gamma) _ ch 56', 'Power ratio (gamma) _ ch 59', 'Med Freq _ ch 54', 'Power ratio (gamma) _ ch 54', 'Power ratio (gamma) _ ch 23', 'Power ratio (gamma) _ ch 33', 'histogram_amp _ ch 9 _ bin 6', 'histogram_am

# Now we predict our test data labels using the best RBF network found with the Fisher method

In [78]:
import pandas as pd

X_test = features_matrix_test[:, Best_rbf_fisher[3]]
print(X_test.shape)
y_pred_rbf = Best_rbf_fisher[4].predict(X_test)

print("Predicted Labels for test set by the best RBF model (Fisher) :\n",np.round(y_pred_rbf).T)
print(y_pred_rbf.shape)

df = pd.DataFrame(y_pred_rbf, columns=['y_pred_rbf_fisher'])
df.to_csv("y_pred_rbf_fisher.csv", index=False)

(159, 50)
Predicted Labels for test set by the best RBF model (Fisher) :
 [0 0 1 1 1 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 1
 1 1 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 1
 1 1 0 0 1 1 0 1 0 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1
 1 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 0 1
 0 1 0 0 1 0 1 1 1 0 1]
(159,)


# PSO

In [60]:
# Sort the combined list by scores in descending order
combined_fisher.sort(key=lambda x: x[1], reverse=True)

# Select the top 50 features
top_1500_features = combined_fisher[:1000]


In [61]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import random

In [62]:
X_features_pso = np.array([feature[2] for feature in top_1500_features]).T
X_features_test_pso = np.array([feature[3] for feature in top_1500_features]).T

print(X_features_test_pso.shape)
print("feature matrix shape = ",X_features_pso.shape)

y_features = y_train

print("label matrix shape = ",y_features.shape)

(159, 1000)
feature matrix shape =  (550, 1000)
label matrix shape =  (550,)


In [63]:
class Particle:
    def __init__(self, n_features):
        self.position = np.random.choice([0.0, 1.0], size=n_features)
        self.velocity = np.random.rand(n_features) * 0.1 
        self.best_position = self.position.copy()
        self.best_score = float('inf')

def evalFeatures(particle_position):
    features = [i for i, bit in enumerate(particle_position) if bit]
    num_features_selected = len(features)

    if num_features_selected == 0:
        return float('inf')

    X_subset = X_features_pso[:, features]
    clf = RandomForestClassifier(n_estimators = 20)
    scores = cross_val_score(clf, X_subset, y_features, cv=5)

    penalty = 0
    ideal_feature_count = 70
    
    if num_features_selected > ideal_feature_count:
        penalty = penalty = 10 * abs(num_features_selected - ideal_feature_count) ** 2

    return -(10*scores.mean()) + penalty

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def update_velocity(particle, global_best_position, w=0.05, c1=10, c2=10):

    r1, r2 = np.random.rand(2, particle.velocity.shape[0])
    cognitive_velocity = c1 * r1 * (particle.best_position.astype(np.float32) - particle.position.astype(np.float32))
    social_velocity = c2 * r2 * (global_best_position.astype(np.float32) - particle.position.astype(np.float32))
    particle.velocity = w * particle.velocity + cognitive_velocity + social_velocity

def update_position(particle, bounds):

     # Use sigmoid to convert velocities to probabilities
    prob = sigmoid(particle.velocity)
    # Update position using a threshold on the probabilities
    particle.position = np.random.rand(len(particle.position)) < prob



In [64]:
def pso(X, y, n_features, n_particles=30, maxiter=100):
    particles = [Particle(n_features) for _ in range(n_particles)]
    global_best_score = float('inf')
    global_best_position = np.zeros(n_features)

    for t in range(maxiter):
        for particle in particles:
            score = evalFeatures(particle.position)

            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = particle.position.copy()

            if score < global_best_score:
                global_best_score = score
                global_best_position = particle.position.copy()

        for particle in particles:
            update_velocity(particle, global_best_position)
            update_position(particle, [0, 1])

    return global_best_position, global_best_score

# Run PSO
n_features = X_features_pso.shape[1]
best_position, best_score = pso(X_features_pso, y_features, n_features)
selected_indices = [i for i, bit in enumerate(best_position > 0.5) if bit]



In [65]:
print("Best score:", best_score)
print("Number of selected features:", len(selected_indices))
selected_features = [top_1500_features[i][0] for i in selected_indices]
print("Selected features:", selected_indices)

df = pd.DataFrame(selected_features, columns=['slected_features_pso'])
df.to_csv("selected_features_pso.csv", index=False)

# Feature Matrix
X_train_pso = X_features_pso[:,selected_indices]


print("X_train shape : ",X_train_pso.shape)
print(type(selected_indices))


y_train = (np.where(train_labels == -1, 0, 1)).reshape(550)

Best score: 1317682.4363636363
Number of selected features: 433
Selected features: [0, 1, 2, 3, 6, 7, 9, 10, 11, 12, 13, 14, 16, 18, 19, 20, 24, 28, 29, 33, 35, 39, 44, 46, 49, 52, 53, 55, 56, 57, 58, 61, 63, 64, 66, 69, 70, 72, 75, 76, 79, 81, 82, 86, 87, 89, 93, 95, 100, 106, 107, 108, 111, 113, 114, 118, 119, 122, 124, 126, 129, 132, 133, 134, 136, 138, 141, 143, 144, 151, 154, 156, 162, 163, 165, 167, 170, 171, 173, 174, 176, 178, 183, 186, 187, 191, 194, 195, 196, 198, 201, 204, 208, 213, 214, 217, 218, 219, 220, 221, 225, 226, 228, 229, 230, 232, 233, 234, 235, 236, 240, 241, 243, 247, 248, 250, 253, 259, 262, 265, 266, 267, 271, 278, 280, 282, 288, 289, 291, 293, 294, 296, 298, 299, 300, 302, 303, 304, 307, 308, 312, 313, 316, 317, 323, 327, 329, 331, 333, 337, 338, 339, 340, 342, 344, 346, 349, 350, 354, 355, 356, 359, 360, 362, 363, 364, 367, 369, 370, 374, 380, 381, 382, 385, 386, 387, 388, 389, 390, 392, 395, 396, 398, 399, 400, 403, 405, 411, 413, 414, 417, 418, 419, 420, 4

# Now we design and train MLP network with the extracted features by tho PSO algorithm

In [66]:
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
from itertools import combinations
import random

In [67]:
# Parameters
n_splits = 5
n_epochs = 100
batch_size = 32
n_features = X_train_pso.shape[1]
accuracy_mlp = 0
acc_train = 0


In [68]:
layers = [[128,64,32,1],[64,16,1]]
mlp_models_pso = []

for i,layer_neurons in enumerate(layers):

    accuracy_mlp=0
   
    model_mlp = create_mlp_model(layer_neurons, n_features)

    # K-Fold 
    kf = KFold(n_splits = n_splits, shuffle = True)
    fold_no = 1

    for train_indeX, test_indeX in kf.split(X_train_pso, y_train):

        X_train_train, X_train_val = X_train_pso[train_indeX], X_train_pso[test_indeX]
        y_train_train, y_train_val = y_train[train_indeX], y_train[test_indeX]


        print(f'Training for fold {fold_no} ...')
        model_mlp.fit(X_train_train, y_train_train, epochs = n_epochs, batch_size = batch_size)


        scores = model_mlp.evaluate(X_train_val, y_train_val, verbose=0)

        print(f'Score for fold {fold_no}: {model_mlp.metrics_names[1]} of {scores[1]*100}%')
        fold_no += 1
        
        accuracy_mlp += scores[1]*100
        

    mlp_models_pso.append((layer_neurons, accuracy_mlp/n_splits, model_mlp))



Training for fold 1 ...
Epoch 1/100
14/14 [==============================] - 0s 1ms/step - loss: 0.6908 - accuracy: 0.5227
Epoch 2/100
14/14 [==============================] - 0s 925us/step - loss: 0.6875 - accuracy: 0.5273
Epoch 3/100
14/14 [==============================] - 0s 876us/step - loss: 0.6616 - accuracy: 0.6295
Epoch 4/100
14/14 [==============================] - 0s 947us/step - loss: 0.6327 - accuracy: 0.6295
Epoch 5/100
14/14 [==============================] - 0s 879us/step - loss: 0.6026 - accuracy: 0.6795
Epoch 6/100
14/14 [==============================] - 0s 891us/step - loss: 0.5652 - accuracy: 0.7205
Epoch 7/100
14/14 [==============================] - 0s 978us/step - loss: 0.6002 - accuracy: 0.6568
Epoch 8/100
14/14 [==============================] - 0s 920us/step - loss: 0.5116 - accuracy: 0.7727
Epoch 9/100
14/14 [==============================] - 0s 1ms/step - loss: 0.4856 - accuracy: 0.7886
Epoch 10/100
14/14 [==============================] - 0s 1ms/step - los

In [69]:
mlp_models_pso.sort(key=lambda x: x[1], reverse=True)

for temp in mlp_models_pso:
    print(f"Layer neurons: {temp[0]}\nAccurcy: {np.round(temp[1],2)}\n")

Layer neurons: [64, 16, 1]
Accurcy: 96.55

Layer neurons: [128, 64, 32, 1]
Accurcy: 95.82



In [70]:
Best_mlp_pso = mlp_models_pso[0]
print(f"Best MLP Model :\n\nLayer neurons: {Best_mlp_pso[0]}\n\nAccurcy: {np.round(Best_mlp_pso[1],2)}%\n")

Best MLP Model :

Layer neurons: [64, 16, 1]

Accurcy: 96.55%



In [80]:
import pandas as pd

X_test = X_features_test_pso[:, selected_indices]
print(X_test.shape)
y_pred_mlp = Best_mlp_pso[2].predict(X_test)

print("Predicted Labels for test set by the best MLP model (PSO) :\n",np.round(y_pred_mlp).T)
print(y_pred_mlp.shape)

df = pd.DataFrame(np.round(y_pred_mlp), columns=['y_pred_mlp_pso'])
df.to_csv("y_pred_mlp_PSO.csv", index=False)

(159, 433)
5/5 [==============================] - 0s 817us/step
Predicted Labels for test set by the best MLP model (PSO) :
 [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0.
  0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1.
  1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0.
  1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1.
  0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0.
  1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1.
  1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1.]]
(159, 1)


# Training RBF with PSO extacted features

In [72]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def rbf(num_centers):

    kfold = KFold(n_splits=n_splits, shuffle=True)
    fold_no = 1
    accuracy_rbf = 0
    svm_classifier_pso = SVC(kernel='rbf', C=num_centers, gamma='scale')  

    for train_indeX, test_indeX in kfold.split(X_train_pso, y_train):
        X_train_train, X_train_val = X_train_pso[train_indeX], X_train_pso[test_indeX]
        y_train_train, y_train_val = y_train[train_indeX], y_train[test_indeX]

    
        # Fit the SVM classifier
        svm_classifier_pso.fit(X_train_train, y_train_train)

        # Predict on the test set
        y_pred = svm_classifier_pso.predict(X_train_val)

        # Calculate accuracy (you can use other metrics as needed)
        scores = accuracy_score(y_train_val, y_pred)
        # print(f'Accuracy for fold {fold_no}: {scores * 100} %')
        accuracy_rbf += (scores * 100)
        fold_no += 1

    # Calculate average accuracy across folds
    avg_accuracy_rbf = accuracy_rbf / n_splits
    print(f'Average Accuracy across 5-fold Cross Validation for RBF network ({num_centers}): {np.round(avg_accuracy_rbf,2)} %')
    
    return ((num_centers, avg_accuracy_rbf, svm_classifier_pso))


In [73]:
import matplotlib.pyplot as plt

num_centers = [16,32,64,128,256,512]
rbf_models_pso = []

for i,num_center in enumerate(num_centers): 
   
    avg_accuracy_rbf = rbf(num_center)
    rbf_models_pso.append(avg_accuracy_rbf)


Average Accuracy across 5-fold Cross Validation for RBF network (16): 88.18 %
Average Accuracy across 5-fold Cross Validation for RBF network (32): 88.73 %
Average Accuracy across 5-fold Cross Validation for RBF network (64): 90.36 %
Average Accuracy across 5-fold Cross Validation for RBF network (128): 86.36 %
Average Accuracy across 5-fold Cross Validation for RBF network (256): 88.91 %
Average Accuracy across 5-fold Cross Validation for RBF network (512): 90.36 %


In [74]:
rbf_models_pso.sort(key=lambda x: x[1], reverse=True)

for temp in rbf_models_pso:
    print(f"Hidden Layer Neuros: {temp[0]}\nAccurcy: {np.round(temp[1],2)}\n")

Hidden Layer Neuros: 64
Accurcy: 90.36

Hidden Layer Neuros: 512
Accurcy: 90.36

Hidden Layer Neuros: 256
Accurcy: 88.91

Hidden Layer Neuros: 32
Accurcy: 88.73

Hidden Layer Neuros: 16
Accurcy: 88.18

Hidden Layer Neuros: 128
Accurcy: 86.36



In [75]:
Best_rbf_pso = rbf_models_pso[0]
print(f"Best RBF Model :\n\nLayer neurons: {Best_rbf_pso[0]}\n\nAccurcy: {np.round(Best_rbf_pso[1],2)}%\n")

Best RBF Model :

Layer neurons: 64

Accurcy: 90.36%



In [79]:
import pandas as pd

X_test = X_features_test_pso[:, selected_indices]
print(X_test.shape)
y_pred_rbf = Best_rbf_pso[2].predict(X_test)

print("Predicted Labels for test set by the best RBF model (PSO) :\n",np.round(y_pred_rbf).T)
print(y_pred_rbf.shape)

df = pd.DataFrame(y_pred_rbf, columns=['y_pred_rbf_pso'])
df.to_csv("y_pred_rbf_pso.csv", index=False)


(159, 433)
Predicted Labels for test set by the best RBF model (PSO) :
 [0 0 1 1 1 0 1 0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 1
 1 1 1 0 1 0 1 0 0 0 1 1 1 1 0 1 1 0 0 1 1 1 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1
 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 1
 0 0 1 0 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1
 0 1 1 0 0 0 1 0 1 0 1]
(159,)


# The End